In [ ]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col,max,count,sum,mean,stddev_pop,hour,countDistinct,expr,stddev,window,column
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
import math

In [2]:
# sc.stop()

# Initialization of SparkConf which is required for Spark Context
conf = SparkConf().setAppName('myapp').setMaster('local')

# Initialization of SparkContext
sc = SparkContext().getOrCreate(conf=conf)
sc.setLogLevel("OFF")

# Initialization of SparkSession into spark variable
spark = SparkSession(sc)

### Reading CSV file from the same folder where this notebook file exists

In [7]:
df = spark.read.csv('summary.csv', inferSchema=True, header=True)
df.printSchema()
#df.dtypes  # may provide wrong solution because it is not from pyspark API

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



### Find the maximum number of flights from Origin country to Destination country by reading flightdata

In [8]:
df.toPandas()

,DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
0,United States,Romania,15
1,United States,Croatia,1
2,United States,Ireland,344
3,Egypt,United States,15
4,United States,India,62
...,...,...,...
251,United States,Saint Kitts and Nevis,145
252,Uruguay,United States,43
253,United States,Haiti,225
254,"Bonaire, Sint Eustatius, and Saba",United States,58


In [9]:
df.select(max(col('count')).alias('Max Flights')).show()

+-----------+
|Max Flights|
+-----------+
|     370002|
+-----------+



### Count those rows where number of flights going from Origin country to Destination country are greater than 10 by reading flightdata again

In [11]:
df.where(col('count') > 10).select(count('*').alias('haha')).show()
df.where(expr('count > 10')).select(expr('count(*) AS haha')).show()
df.where('count > 10').count()
df.filter('count > 10').count()

+----+
|haha|
+----+
| 208|
+----+

+----+
|haha|
+----+
| 208|
+----+



208

### Count total number of flights having destination country name is United States

In [12]:
df.where(col('DEST_COUNTRY_NAME') == 'United States').agg(sum('count').alias('sum')).show()
df.groupBy('DEST_COUNTRY_NAME').agg(sum('count').alias('sum')).where("DEST_COUNTRY_NAME = 'United States'").show()
df.filter("DEST_COUNTRY_NAME = 'United States'").agg(sum('count')).show()
df.filter("DEST_COUNTRY_NAME = 'United States'").agg({"count" : "sum"}).show()

+------+
|   sum|
+------+
|411352|
+------+

+-----------------+------+
|DEST_COUNTRY_NAME|   sum|
+-----------------+------+
|    United States|411352|
+-----------------+------+

+----------+
|sum(count)|
+----------+
|    411352|
+----------+

+----------+
|sum(count)|
+----------+
|    411352|
+----------+

